# Simple correlation Project - data sourced from Google drive spreadsheet


This notebook presents a simple example of code for reading and writing data to a spreadsheet on Google drive.
It is needed to share results to coleagues who would like to use spreadsheets only, rather than Python dataframes


It is necessary to enable the permission in the drive of course for the spreadsheets you access, or this code will fail. 

A simple exercise is illustrated: reading data. Appending new data to those. And writing new data to another sheet.

## Import libraries

In [1]:
!pip install gspread
!pip install oauth2client

     |████████████████████████████████| 151 kB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 3.7 MB/s eta 0:00:011
     |████████████████████████████████| 96 kB 4.7 MB/s eta 0:00:011


In [5]:
# libraries to read data
import numpy as np
import pandas as pd
# libraries to model data

# libraries to connect to your google spreadsheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials


## Collect data from the google spreadsheet

In [6]:
def get_historicalData():
    #Create the connection to the the desired Google Sheet and insert into a dataframe
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    # credentials are stored in the file that you downloaded from the console
    creds = ServiceAccountCredentials.from_json_keyfile_name('keys/project-x-brussels-f1632aa573cd.json', scope)
    client = gspread.authorize(creds)
    # here you need to insert the ID of the G-Sheets file. That can be copied from the navigation bar in your browser 
    sheetfile =  client.open_by_key('1_xKwVFweaXgAMwN9tPtccTPfxgUvCKM-3ViSp5c3LpM')
    
    sheet = sheetfile.get_worksheet(0) # first worksheet in the G-Sheets file
    data = sheet.get_all_values()
    headers = data.pop(0)
    df = pd.DataFrame(data, columns=headers)
    return df

In [7]:
# create a Pandas dataframe from Google sheets

df_hist = get_historicalData()

In [10]:
df_hist.tail()

,date,rate confirmed,acc_test,acc_negative,acc_confirmed,acc_released,acc_deceased,new_test,new_negative,new_confirmed,new_released,new_deceased
39,02/28/20,813,81167,48593,2337,27,13,14515,9275,571,1,0
40,02/29/20,586,94055,55723,3150,28,17,12888,7130,813,1,4
41,03/01/20,476,98921,61825,3736,30,18,4866,6102,586,2,1
42,03/02/20,600,109591,71580,4212,31,22,10670,9755,476,1,4
43,03/03/20,516,125851,85484,4812,34,28,16260,13904,600,3,6


## Append data from a file on your local drive

In [14]:
# get data to append from a local file
df_recentdata = pd.read_csv('logfile.csv').fillna(0)
df_recentdata.head()

,date,rate confirmed,acc_test,acc_negative,acc_confirmed,acc_released,acc_deceased,new_test,new_negative,new_confirmed,new_released,new_deceased
0,03/04/20,438,136707,102965,5328,41,32,10856,17481,516,7,4
1,03/05/20,518,146541,118965,5766,88,35,9834,16000,438,47,3


In [15]:
spreadsheetId = '1_xKwVFweaXgAMwN9tPtccTPfxgUvCKM-3ViSp5c3LpM'  # Please set the Spreadsheet ID.
sheetName = 'data'

def append_DFvalues_gspreadsheet(df, spreadsheetId, sheetName):
  scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
  creds = ServiceAccountCredentials.from_json_keyfile_name('keys/project-x-brussels-f1632aa573cd.json', scope)
  client = gspread.authorize(creds)
  sh = client.open_by_key(spreadsheetId)

  values = df.values.tolist()
  return sh.values_append(sheetName, {'valueInputOption': 'USER_ENTERED'}, {'values': values})

In [16]:
append_DFvalues_gspreadsheet(df_recentdata, spreadsheetId, sheetName)

{'spreadsheetId': '1_xKwVFweaXgAMwN9tPtccTPfxgUvCKM-3ViSp5c3LpM',
 'tableRange': 'data!A1:L45',
 'updates': {'spreadsheetId': '1_xKwVFweaXgAMwN9tPtccTPfxgUvCKM-3ViSp5c3LpM',
  'updatedRange': 'data!A46:L47',
  'updatedRows': 2,
  'updatedColumns': 12,
  'updatedCells': 24}}

In [23]:
df_stats = pd.DataFrame(df_hist['date'].tolist(), columns = ['date'])


In [29]:
df_stats ['rate_infection'] = df_hist['new_confirmed'].astype('int')/df_hist['new_test'].astype('int')
df_stats = df_stats.fillna(0)

In [30]:
def update_DFvalues_gspreadsheet(df, spreadsheetId, sheetName):
  scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
  creds = ServiceAccountCredentials.from_json_keyfile_name('keys/project-x-brussels-f1632aa573cd.json', scope)
  client = gspread.authorize(creds)
  sh = client.open_by_key(spreadsheetId)

  values = df.values.tolist()
  values[0] = df.columns.values.tolist() #add headers

  return sh.values_update(sheetName, {'valueInputOption': 'USER_ENTERED'}, {'values': values})


In [31]:
sheetName = 'metrics'
update_DFvalues_gspreadsheet(df_stats, spreadsheetId, sheetName)

{'spreadsheetId': '1_xKwVFweaXgAMwN9tPtccTPfxgUvCKM-3ViSp5c3LpM',
 'updatedRange': 'metrics!A1:B44',
 'updatedRows': 44,
 'updatedColumns': 2,
 'updatedCells': 88}

In [32]:
df_stats.values.tolist()

[['01/20/20', 1.0],
 ['01/21/20', 0.0],
 ['01/22/20', 0.0],
 ['01/23/20', 0.0],
 ['01/24/20', 0.2],
 ['01/25/20', 0.0],
 ['01/26/20', 0.041666666666666664],
 ['01/27/20', 0.1],
 ['01/28/20', 0.0],
 ['01/29/20', 0.0],
 ['01/30/20', 0.03389830508474576],
 ['01/31/20', 0.07575757575757576],
 ['02/01/20', 0.01694915254237288],
 ['02/02/20', 0.05172413793103448],
 ['02/03/20', 0.0],
 ['02/04/20', 0.008547008547008548],
 ['02/05/20', 0.018691588785046728],
 ['02/06/20', 0.029239766081871343],
 ['02/07/20', 0.0021413276231263384],
 ['02/08/20', 0.0],
 ['02/09/20', 0.005988023952095809],
 ['02/10/20', 0.0],
 ['02/11/20', 0.0008230452674897119],
 ['02/12/20', 0.0],
 ['02/13/20', 0.0],
 ['02/14/20', 0.0],
 ['02/15/20', 0.0],
 ['02/16/20', 0.00234192037470726],
 ['02/17/20', 0.0017953321364452424],
 ['02/18/20', 0.0009487666034155598],
 ['02/19/20', 0.014275517487508922],
 ['02/20/20', 0.026121241991128634],
 ['02/21/20', 0.031269543464665414],
 ['02/22/20', 0.04415734670266101],
 ['02/23/20', 0.